# Эксперимент по определению авторства текста.

In [ ]:
!pip install torch --index-url https://download.pytorch.org/whl/cu124
!pip install datasets transformers accelerate scikit-learn evaluate langchain tensorboardX

In [ ]:
# @title подгрузим файлики с диска
from pathlib import Path

#@markdown gt_texts - массив авторских текстов: берем все файлики из каталога "ground_truth"
gt_texts = []
p = Path("ground_truth")
for x in p.rglob("*"):
    with open(x, 'r', encoding='utf-8') as file:
        gt_texts.append(file.read())

#@markdown others - массив текстов других авторов: берем все файлики из каталога "other"
others = []
p = Path("other")
for x in p.rglob("*"):
    with open(x, 'r', encoding='utf-8') as file:
        others.append(file.read())

#@markdown gt_texts - массив текстов, среди которых надо определить автора, каталог: "target_corp"
targets = []
p = Path("target_corp")
for x in p.rglob("*"):
    with open(x, 'r', encoding='utf-8') as file:
        targets.append(file.read())

In [ ]:
# @title нарежем обучающие данные на чанки небольших размеров с перекрытием.
from datasets import Dataset
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

gt_chunks = text_splitter.create_documents(gt_texts)
oth_chunks = text_splitter.create_documents(others)

#@markdown и соберем датасет
my_ds = Dataset.from_list(
    [{"label": 1, "text": x.page_content} for x in gt_chunks]
    + [{"label": 0, "text": x.page_content} for x in oth_chunks]
)

my_ds = my_ds.class_encode_column("label")
my_ds = my_ds.train_test_split(test_size=0.1,stratify_by_column='label', seed=123)
my_ds


Stringifying the column:   0%|          | 0/282 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/282 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 253
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 29
    })
})

In [ ]:
# @title метрики
import numpy as np
import evaluate

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return clf_metrics.compute(predictions,labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# @title модель и токенизация датасета

from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'deepvk/USER-base'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
print(model)

# @markdown учим только голову
for p in model.deberta.parameters():
    p.requires_grad = False

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = my_ds['train'].map(tokenize_function, batched=True)
train_dataset = tokenized_dataset.shuffle(seed=42)

tokenized_ev_dataset = my_ds['test'].map(tokenize_function, batched=True)
eval_dataset = tokenized_ev_dataset.shuffle(seed=42)



Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at deepvk/USER-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaForSequenceClassification(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (intermediate): DebertaIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermediate_act_fn): GELUActivat

Map:   0%|          | 0/253 [00:00<?, ? examples/s]

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [ ]:
# @title Параметры обучения
from datetime import datetime
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="test_trainer", #The output directory
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy= "epoch",
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=100, # number of training epochs
    #learning_rate = 2e-5,
    lr_scheduler_type = 'cosine',
    #eval_steps = 10, # Number of update steps between two evaluations.
    #logging_steps = 100,  # Number of update steps between two evaluations.
    logging_dir = f'logs/{datetime.now().strftime("%d_%m_%H.%M.%S")}',
    warmup_steps = 100, # number of warmup steps for learning rate scheduler
    #weight_decay = 0.2,
    report_to='tensorboard',
    metric_for_best_model='eval_loss',
    load_best_model_at_end=True,
    greater_is_better=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train() #@markdown понеслась

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.675800,0.671601,0.551724,0.000000,0.000000,0.000000
2,0.661400,0.645851,0.655172,0.375000,1.000000,0.230769
3,0.638000,0.612294,0.655172,0.375000,1.000000,0.230769
4,0.590900,0.569602,0.827586,0.761905,1.000000,0.615385
5,0.538300,0.527014,0.793103,0.700000,1.000000,0.538462
6,0.496800,0.497185,0.793103,0.700000,1.000000,0.538462
7,0.460000,0.472075,0.793103,0.700000,1.000000,0.538462
8,0.420100,0.448690,0.793103,0.700000,1.000000,0.538462
9,0.385300,0.435343,0.793103,0.700000,1.000000,0.538462
10,0.358100,0.421479,0.793103,0.700000,1.000000,0.538462


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=512, training_loss=0.427614439278841, metrics={'train_runtime': 246.7519, 'train_samples_per_second': 102.532, 'train_steps_per_second': 12.968, 'total_flos': 1064958946934784.0, 'train_loss': 0.427614439278841, 'epoch': 16.0})

In [ ]:
# @title Ищем текст автора...
import torch as pt

for x in targets:
    text = x
    inpt = tokenizer.encode(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    model.cuda()
    model.eval()
    inpt= inpt.cuda()
    out = model(inpt)
    it = pt.argmax(out[0], dim=1).item()
    m = pt.nn.Softmax(dim=1)
    prc = pt.max(m(out[0])).item()*100
    #s = ans[it]
    print(f'{text[:50]} : {it} (Вероятность:{prc:.2f}%)')

Цветные пятна яркими мотыльками мельтешили перед н : 0 (Вероятность:56.89%)
Деревня скрылась в тёмных покрывалах ночи. Склон х : 0 (Вероятность:50.80%)
Библиотекарь и начинающий писатель Ваня Вестный от : 1 (Вероятность:52.06%)
Когда круги перестали плыть перед глазами, начинаю : 0 (Вероятность:58.64%)
Когда Ваня Вестный, начинающий писатель, библиотек : 0 (Вероятность:56.26%)
 Эх, не те книжки читал библиотекарь Ваня, он же н : 0 (Вероятность:54.18%)
Что-то мягкое и прохладное касалось Ваниного лица. : 0 (Вероятность:53.19%)
Начинающий писатель Иван Вестный вновь очутился в  : 0 (Вероятность:57.06%)


Важная ремарка:

Очень мало было данных для обучения и результат оказался не очень воспроизводимым!

В данном случае мне повезло и нейросеть действительно попала в правильный ответ, причем однозначно, но в последующих экспериментах были случаи, где более одного текста считались текстами автора (хотя все-таки чаще и более низким коэффицентом, чем действительно авторский).

В общем сильно зависит от разделения тест/трейн,ч что не удивительно на таком малом корпусе.